In [7]:
from torchvision.transforms import RandomCrop, Compose, ToPILImage, Resize, ToTensor, Lambda
from med_ddpm.diffusion_model.trainer import GaussianDiffusion, Trainer
from med_ddpm.diffusion_model.unet import create_model
import argparse
import torch
import os 
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset
from torchvision.transforms import Compose, ToTensor, Lambda
from glob import glob
import matplotlib.pyplot as plt
import nibabel as nib
import torchio as tio
import numpy as np
import torch
from tqdm import tqdm
import re
import os
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
params = {
    "with_condition": True,
    "inputfolder": "../../data/registration_data/DWI/",
    "targetfolder": "../../data/registration_data/CT/",
    "batchsize": 1,
    "epochs": 10000,
    "input_size": 256,
    "depth_size": 64,
    "num_channels": 64,
    "num_res_blocks": 1,
    "timesteps": 250,
    "save_and_sample_every": 100,
    "model_save_path": "../../model/med_ddpm_translation/dwi2ct/",
    "resume_weight": "../../model/med_ddpm/mri/model-17.pt"
}

In [ ]:
transform = Compose([
    Lambda(lambda t: torch.tensor(t).float()),
    Lambda(lambda t: (t * 2) - 1),
    Lambda(lambda t: t.unsqueeze(0))
])
class NiftiPairImageGenerator(Dataset):
    def __init__(self,
            input_image,
            target_image,
            input_size: int,
            depth_size: int,
            input_channel: int = 3,
            target_transform=None,
            full_channel_mask=False,
            combine_output=False
        ):

        self.input_image = input_image
        self.target_image = target_image
        self.input_size = input_size
        self.depth_size = depth_size
        self.input_channel = input_channel
        self.scaler = MinMaxScaler()
        self.target_transform = target_transform
        self.full_channel_mask = full_channel_mask
        self.combine_output = combine_output


    def plot(self, index, n_slice=30):
        data = self[index]
        input_img = data['input']
        target_img = data['target']
        plt.subplot(1, 2, 1)
        plt.imshow(input_img[:, :, n_slice])
        plt.subplot(1, 2, 2)
        plt.imshow(target_img[:, :, n_slice])
        plt.show()

    def __len__(self):
        return len(self.pair_files)

    def __getitem__(self, index):
        input_img = self.input_image[index]
        target_img = self.target_image[index]
      

        if self.combine_output:
            return torch.cat([target_img, input_img], 0)

        return {'input':input_img, 'target':target_img}

def resize_img(img):
    d,h, w = img.shape
    if h !=  params['input_size'] or w !=  params['input_size'] or d != params['depth_size']:
        img = tio.ScalarImage(tensor=img[np.newaxis, ...])
        cop = tio.Resize((params['depth_size'], params['input_size'],  params['input_size']))
        img = np.asarray(cop(img))[0]
    return img 
input_list=glob(params['inputfolder']+"*.nii.gz")
target_list=[f.replace(params['inputfolder'], params['targetfolder']) for f in input_list]
input_images=torch.zeros((len(input_list), params['depth_size'], params['input_size'], params['input_size']))
target_images=torch.zeros((len(target_list), params['depth_size'], params['input_size'], params['input_size']))
for i in tqdm(range(len(input_list))):
    input_img = nib.load(input_list[i]).get_fdata()
    target_img = nib.load(target_list[i]).get_fdata()
    input_img = (input_img - np.min(input_img)) / (np.max(input_img) - np.min(input_img) + 1e-8)
    target_img = (target_img - np.min(target_img)) / (np.max(target_img) - np.min(target_img) + 1e-8)
    input_images[i] =transform(resize_img(input_img))
    target_images[i] = transform(resize_img(target_img))
    
dataset = NiftiPairImageGenerator(
        input_images,
        target_images,
        input_size=params['input_size'],
        depth_size=params['depth_size'],
        target_transform=transform,
        full_channel_mask=True
    )

100%|██████████| 724/724 [03:57<00:00,  3.04it/s]


In [ ]:

dataset = NiftiPairImageGenerator(
        input_images,
        target_images,
        input_size=params['input_size'],
        depth_size=params['depth_size'],
        target_transform=transform,
        full_channel_mask=True
    )
dataset[0]['target'].max()

tensor(1.)

In [4]:
in_channels=1
out_channels=1
model = create_model(params["input_size"], params["num_channels"], params["num_res_blocks"], in_channels=in_channels, out_channels=out_channels).to(device)

diffusion = GaussianDiffusion(
    model,
    image_size = params["input_size"],
    depth_size = params["depth_size"],
    timesteps = params["timesteps"],   # number of steps
    loss_type = 'l1',    # L1 or L2
    with_condition=params["with_condition"],
    channels=out_channels
).to(device)